# Random Forest Classifier

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('./dataset/airbnb_cleaned.csv', encoding='Latin-1')

In [4]:
df['zipcode'] = pd.to_numeric(df['zipcode'], errors='coerce').fillna(0)
df['host_is_superhost'] = pd.to_numeric(df['host_is_superhost'], errors='coerce').fillna(0)

In [5]:
#oh_propertytype = pd.get_dummies(df['property_type'])
#df = df.drop('property_type',axis = 1)
#df = df.join(oh_propertytype)

cleanup_proptype = {'Apartment': 1, 'House': 2, 'Townhouse': 3, 'Loft': 4, 'Condominium': 5, 'Serviced apartment': 6,  'Guest suite': 7, 'Other': 8,
                     'Bed and breakfast': 9, 'Bed & Breakfast': 9, 'Boutique hotel': 10, 'Resort': 11, 'Guesthouse': 12, 'Hostel': 13, 'Hotel': 14, 'Bungalow': 15,
                    'Villa': 16, 'Tiny house': 17, 'Aparthotel': 18, 'Boat': 19, 'Cottage': 20, 'Tent': 21, 'Camper/RV': 22, 'Cabin': 23, 'Casa particular (Cuba)': 24,
                   'Castle': 25, 'Houseboat': 26, 'Earth house': 27, 'Nature lodge': 28, 'Cave': 29, 'Train': 30, 'Timeshare': 31, 'Island': 32, 'Bus': 33, 'Chalet': 34
                   }
df['property_type'].replace(cleanup_proptype, inplace=True)
df.head()
df

,Unnamed: 0,id,host_id,zipcode,neighbourhood_cleansed,property_type,room_type,accommodates,bedrooms,beds,bed_type,price,number_of_reviews,review_scores_rating,availability_30,minimum_nights,bathrooms,host_is_superhost,host_response_rate
0,0,2515,2758,10026.0,Harlem,1,1.01,3,1.0,2.0,1.00,59.0,173,93.0,10,2,1.0,0.0,0.0
1,1,2539,2787,11218.0,Kensington,1,1.01,4,1.0,3.0,1.00,149.0,8,98.0,25,1,1.0,1.0,0.0
2,2,2595,2845,10018.0,Midtown,1,1.00,2,0.0,1.0,1.00,225.0,36,94.0,20,1,1.0,0.0,80.0
3,3,3330,4177,11206.0,Williamsburg,1,1.01,2,1.0,1.0,1.00,70.0,34,97.0,1,5,1.0,0.0,0.0
4,4,3647,4632,10027.0,Harlem,1,1.01,2,1.0,1.0,1.02,150.0,0,NaN,30,3,1.0,0.0,50.0
5,5,3831,4869,11238.0,Clinton Hill,7,1.00,3,1.0,3.0,1.00,89.0,213,91.0,0,1,1.0,0.0,100.0
6,7,5099,7322,10016.0,Murray Hill,1,1.00,2,1.0,1.0,1.00,185.0,66,87.0,4,2,1.0,0.0,100.0
7,9,5172,7386,10016.0,Murray Hill,1,1.00,2,1.0,1.0,1.00,129.0,82,94.0,0,3,1.0,1.0,0.0
8,10,5178,8967,10019.0,Hell's Kitchen,1,1.01,2,1.0,1.0,1.00,79.0,396,84.0,2,2,1.0,0.0,91.0
9,11,5203,7490,10025.0,Upper West Side,1,1.01,1,1.0,1.0,1.00,79.0,118,98.0,0,2,1.0,0.0,0.0


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [7]:
def LRfindAveragePrice(location):
    return (df[(df.neighbourhood_cleansed == location)])

area = LRfindAveragePrice('Allerton')
area.drop(['neighbourhood_cleansed'], axis=1)

,Unnamed: 0,id,host_id,zipcode,property_type,room_type,accommodates,bedrooms,beds,bed_type,price,number_of_reviews,review_scores_rating,availability_30,minimum_nights,bathrooms,host_is_superhost,host_response_rate
555,591,182177,873273,10469.0,3,1.00,8,1.0,2.0,1.0,125.0,255,93.0,14,2,1.0,0.0,100.0
1271,1344,498120,2459648,10469.0,2,1.01,1,2.0,1.0,1.0,35.0,2,100.0,20,7,1.0,0.0,0.0
1754,1851,715270,3684360,10467.0,1,1.01,3,1.0,2.0,1.0,39.0,155,90.0,11,2,1.0,0.0,98.0
1858,1961,755528,3684360,10467.0,1,1.00,3,1.0,2.0,1.0,49.0,153,90.0,1,1,1.0,0.0,98.0
1870,1973,759901,873273,10469.0,2,1.01,2,1.0,1.0,1.0,85.0,37,94.0,20,5,1.5,0.0,100.0
1912,2016,773041,3684360,10467.0,1,1.01,3,1.0,2.0,1.0,38.0,139,88.0,3,1,1.0,0.0,98.0
5513,5694,3400359,16286162,10469.0,2,1.01,1,1.0,1.0,1.0,45.0,103,93.0,13,2,1.5,1.0,100.0
5550,5731,3429765,16286162,10469.0,2,1.01,1,1.0,1.0,1.0,45.0,75,96.0,0,2,1.0,1.0,100.0
7120,7319,4462008,16286162,10469.0,2,1.01,2,1.0,2.0,1.0,43.0,92,91.0,0,2,1.0,1.0,100.0
8238,8451,5046189,23732730,10469.0,2,1.00,4,0.0,3.0,1.0,200.0,44,96.0,30,30,1.0,1.0,100.0


In [8]:
df_features = area[['accommodates', 'beds', 'bedrooms', 'bathrooms', 'availability_30', 'minimum_nights', 'number_of_reviews', 'bed_type', 'host_response_rate', 'room_type', 'zipcode', 'property_type']]
df_target = area['price']

In [9]:
feat_labels = ['accommodates', 'beds', 'bedrooms', 'bathrooms', 'availability_30', 'minimum_nights', 'number_of_reviews', 'bed_type', 'host_response_rate', 'host_is_superhost', 'room_type', 'zipcode', 'property_type']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.3, random_state=0)

In [11]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=20000, random_state=0, n_jobs=-1)

# Train the classifier
X_test.fillna(X_test.mean())
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feat_labels, clf.feature_importances_):
    print(feature)


('accommodates', 0.11104042521793883)
('beds', 0.05065415184236624)
('bedrooms', 0.06746341245812393)
('bathrooms', 0.05159995338684846)
('availability_30', 0.12975145093416657)
('minimum_nights', 0.12365476859329394)
('number_of_reviews', 0.17859777334668528)
('bed_type', 0.0)
('host_response_rate', 0.08116558834347207)
('host_is_superhost', 0.05202740859307637)
('room_type', 0.07041436998652534)
('zipcode', 0.08363069729750326)


In [12]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance >= threshold
sfm = SelectFromModel(clf, threshold=0.07)

# Train the selector
sfm.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=0.07)

In [13]:
# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

accommodates
availability_30
minimum_nights
number_of_reviews
host_response_rate
room_type
zipcode


In [14]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

In [15]:
# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=20000, random_state=0, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [16]:
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.7

In [19]:
import statsmodels.api as sm

df_features = area[['accommodates', 'availability_30', 'minimum_nights', 'number_of_reviews','host_response_rate', 'zipcode']]
df_predictors = sm.add_constant(df_features)
est = sm.OLS(df_target, 
            df_features).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     111.8
Date:                Fri, 17 May 2019   Prob (F-statistic):           4.79e-40
Time:                        00:58:40   Log-Likelihood:                -493.95
No. Observations:                  98   AIC:                             999.9
Df Residuals:                      92   BIC:                             1015.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
accommodates          14.7493      1